In [1]:
import pandas as pd

In [2]:
os.chdir('dataframes_pkl')

In [5]:
pd.read_pickle('df_global_format.pkl')['Element'].unique()

array(['Export Quantity', 'Export Value', 'Import Quantity',
       'Import Value', 'Production', 'Loss', 'Processed',
       'Food supply quantity (tonnes)', 'Feed', 'Seed',
       'Other uses (non-food)', 'Tourist consumption',
       'Emissions intensity', 'Emissions (CO2eq)',
       'Total Population - Both sexes', 'Rural population',
       'Urban population', 'Export Transport Emissions Quantity',
       'Import Transport Emissions Quantity'], dtype=object)